In [114]:
# import stuff here
import requests
from bs4 import BeautifulSoup

In [115]:
page = requests.get("https://redplanetscience.com/")
page

<Response [200]>

In [116]:
# NASA Mars News
# Scrape the Mars News Site, collect latest News Title and Paragraph Text. 
#soup = BeautifulSoup(page.content, 'html.parser')
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet">
   <title>
    News - Mars Exploration Program
   </title>
  </link>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <s

In [117]:
#list(soup.children)

In [118]:
title = soup.title.text
print(title)

News - Mars Exploration Program


In [126]:
# body = soup.body.text
# body
type(soup)
soup.

bs4.BeautifulSoup

In [122]:
title = soup.find_all('div', class_='content_title').find('a').text

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [107]:
results = soup.find_all('section', class_='image_and_description_container')
print(len(results))
soup.body.section
sections = soup.find_all('section', class_='image_and_description_container')

for section in sections:
    print(section.find('div', class_='article_teaser_body'))

1
None


In [66]:
for result in results:
    #print(result)
    try:
        #title = result.find('div', class_='content_title').text
        title = 
        blurb = soup.find_all('div', class_='article_teaser_body')
        
        if (title and blurb):
            print('------------')
            print(title)
            print(blurb)
    except AttributeError as e:
        print(e)

'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'
'NoneType' object has no attribute 'text'


In [ ]:
headlines = []

for div in divs:
    if (div.)

In [43]:

divList = soup.body.find_all('section', class_='image_and_description_container')
len(divList)

1

In [41]:
# paragraphs = soup.find_all('section', class_='image_and_description_container')
# for paragraph in paragraphs:
#     print(paragraph.text)

In [42]:
results = soup.find_all('div', class_='list_text')
for result in results:
    print(result)

In [ ]:
#list(soup.children)
print(soup.prettify())

In [ ]:
[type(item) for item in list(soup.children)]

In [ ]:
html = list(soup.children)[2]
html

In [ ]:
list(html.children)

In [ ]:
body = list(html.children)[3]
body

In [ ]:
p = list(body.children)[1]

# for kid in p:
#     p.get_text()

In [ ]:
p.get_text()

In [ ]:
soup.find_all('p')[0].get_text()

In [ ]:
x = soup.find_all('p')
x

In [ ]:
# JPL Mars Space Images - Featured Image